# Question Answering Model 
## no trainer

- dataset
- torch
- transformers
- transformers[torch]
- evaluate

import packages

In [2]:
from datasets import load_dataset, disable_caching
from transformers import (
    AutoTokenizer,
    default_data_collator,
    get_scheduler,
    AutoModelForQuestionAnswering,
    BertConfig
)
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch

import evaluate
import collections
from tqdm.auto import tqdm
import numpy as np
from sklearn.cluster import KMeans

import os
import re
import datetime

from torch.autograd import Variable
import torch.nn.functional as F
from copy import deepcopy

Set cache directory.

In [4]:
CACHE_DIR='/mount/arbeitsdaten31/studenten1/linku/.cache'
%set_env TRANSFORMERS_CACHE $CACHE_DIR
%set_env HF_MODULES_CACHE $CACHE_DIR
%set_env HF_DATASETS_CACHE $CACHE_DIR

env: TRANSFORMERS_CACHE=/mount/arbeitsdaten31/studenten1/linku/.cache
env: HF_MODULES_CACHE=/mount/arbeitsdaten31/studenten1/linku/.cache
env: HF_DATASETS_CACHE=/mount/arbeitsdaten31/studenten1/linku/.cache


### arguments.py

args_input.

In [10]:
args_input_ALstrategy = 'RandomSampling'
# args_input_initseed = 100 # 1000
args_input_quota = 200 # 1000
args_input_batch = 50 # 128
args_input_dataset_name = 'DROP'
args_input_iteration = 5
args_input_model_batch = 8 # already add in arguments.py
args_input_max_length = None

stride = 128

in main.py

In [6]:
MAX_LENGTH = args_input_max_length
NUM_QUERY = args_input_batch
# NUM_INIT_LB = args_input_initseed
NUM_ROUND = int(args_input_quota / args_input_batch)
DATA_NAME = args_input_dataset_name
STRATEGY_NAME = args_input_ALstrategy

In [8]:
pretrain_model_dir = '/mount/arbeitsdaten31/studenten1/linku/pretrain_models' + '/' + 'RoBERTa_SQuAD_full_dataset'

### load dataset

In [3]:
disable_caching()

In [11]:
def load_dataset_mrqa(d):
	'''
	return train_set, val_set
	'''
	data = load_dataset("mrqa", cache_dir=CACHE_DIR)
	if d == 'squad':
		# the first to 86588th in train set
		# the first to 10507th in val set		
		return data['train'].select(range(86588)), data['validation'].select(range(10507))
	elif d == 'newsqa':
		# the 86589th to 160748th in train set
		# the 10508th to 14719th in val set		
		return data['train'].select(range(86588, 160748)), data['validation'].select(range(10507, 14719))
	elif d == 'searchqa':
		# the 222437th to 339820th in train set
		# the 22505th to 39484th in val set
		return data['train'].select(range(222436, 339820)), data['validation'].select(range(22504, 39484))
	elif d == 'bioasq':
		# the first to the 1504th in the test set
		sub = data['test'].select(range(1504))
		len_sub_val = len(sub) // 10
		return sub.select(range(len_sub_val, len(sub))), sub.select(range(len_sub_val))
	elif d == 'textbookqa':
		# the 8131st to 9633rd
		sub = data['test'].select(range(8130, 9633))
		len_sub_val = len(sub) // 10
		return sub.select(range(len_sub_val, len(sub))), sub.select(range(len_sub_val)) 
	elif d == 'drop': # Discrete Reasoning Over Paragraphs
		# the 1505th to 3007th in test set
		sub = data['test'].select(range(1504, 3007))
		len_sub_val = len(sub) // 10
		return sub.select(range(len_sub_val, len(sub))), sub.select(range(len_sub_val))

In [14]:
# TODO: function different!!!

# train_data, val_data = load_dataset_mrqa(DATA_NAME.lower())
train_data, val_data = load_dataset_mrqa('squad')
squad = load_dataset('squad', cache_dir=CACHE_DIR)

Found cached dataset mrqa (/mount/arbeitsdaten31/studenten1/linku/.cache/mrqa/plain_text/1.1.0/1f2cf5ac32b43f864e6f91d384057a16b69b7d13ba9bcaa200ac277c90938d19)


  0%|          | 0/3 [00:00<?, ?it/s]

Found cached dataset squad (/mount/arbeitsdaten31/studenten1/linku/.cache/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
train_data = train_data.select(range(100))
val_data = val_data.select(range(10))
squad_train_data = squad['train'].select(range(100))
squad_val_data = squad['validation'].select(range(10))

Next we will preprocess the dataset (training and evaluation data).

In [32]:
def preprocess_training_examples(examples, tokenizer):
    # no ['offset_mapping'], for .train() and .eval()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        # print('answer["answer_start"]:', answer)
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["id"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [24]:
def preprocess_training_examples_lowRes(examples, tokenizer):
    # no ['offset_mapping'], for .train() and .eval()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    detected_answers = examples["detected_answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = detected_answers[sample_idx]
        start_char = answer["char_spans"][0]["start"][0]
        end_char = answer["char_spans"][0]["end"][0]
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["qid"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [31]:
def preprocess_training_features(examples, tokenizer):
    # keep ["offset_mapping"] and ["example_id"], for compute_metrics()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        # print('answer["answer_start"]:', answer)
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["id"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [25]:
def preprocess_training_features_lowRes(examples, tokenizer):
    # keep ["offset_mapping"] and ["example_id"], for compute_metrics()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs["offset_mapping"]
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["detected_answers"]
    start_positions = []
    end_positions = []
    example_ids = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        # print('answer["char_spans"]:', answer)
        start_char = answer["char_spans"][0]["start"][0]
        end_char = answer["char_spans"][0]["end"][0]
        sequence_ids = inputs.sequence_ids(i)
        
        example_ids.append(examples["qid"][sample_idx]) # newly added for used in unlabel data predict

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["example_id"] = example_ids
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [21]:
def preprocess_validation_examples(examples, tokenizer):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [22]:
def preprocess_validation_examples_lowRes(examples, tokenizer):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=MAX_LENGTH,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["qid"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [16]:
# load tokenizer for dataset preprocessing
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [26]:
# TODO: The data name different!!!

# preprocess data
train_dataset = train_data.map(
    preprocess_training_examples_lowRes,
    batched=True,
    remove_columns=train_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)
train_features = train_data.map(
    preprocess_training_features_lowRes,
    batched=True,
    remove_columns=train_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)
val_dataset = val_data.map(
    preprocess_validation_examples_lowRes,
    batched=True,
    remove_columns=val_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)
val_features = val_data.map(
    preprocess_validation_examples_lowRes,
    batched=True,
    remove_columns=val_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [33]:
# origin squad preprocess data
squad_train_dataset = squad_train_data.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=squad_train_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)
squad_train_features = squad_train_data.map(
    preprocess_training_features,
    batched=True,
    remove_columns=squad_train_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)
squad_val_dataset = squad_val_data.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad_val_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)
squad_val_features = squad_val_data.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad_val_data.column_names,
    fn_kwargs=dict(tokenizer=tokenizer)
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [34]:
train_dataset.set_format("torch")
train_features.set_format("torch")
val_dataset = val_dataset.remove_columns(["offset_mapping"])
val_dataset.set_format("torch")
val_features.set_format("torch")

In [35]:
squad_train_dataset.set_format("torch")
squad_train_features.set_format("torch")
squad_val_dataset = squad_val_dataset.remove_columns(["offset_mapping"])
squad_val_dataset.set_format("torch")
squad_val_features.set_format("torch")

## model.py

In [ ]:
strategy_model_dir = model_dir + '/lowRes_' + str(args_input.quota) + '_' + STRATEGY_NAME + '_' + MODEL_NAME +  '_' + DATA_NAME

In [14]:
def to_train(num_train_epochs, train_dataloader, device, model, optimizer, lr_scheduler, record_loss=False):
	print('Training was performed using the sum of {} initial data and {} query data, i.e. {} data.'.format(NUM_INIT_LB, NUM_QUERY, len(train_dataloader.dataset)))
	for epoch in range(num_train_epochs):
		model.train()
		for step, batch in enumerate(tqdm(train_dataloader, desc="Training")):
			batch = {key: value.to(device) for key, value in batch.items()}
			outputs = model(**batch)
			loss = outputs.loss
			loss.backward()

			optimizer.step()
			lr_scheduler.step()
			optimizer.zero_grad()

		if record_loss:
			print('Train Epoch: {}\tLoss: {:.6f}'.format(epoch, loss.item()))

	model_to_save = model.module if hasattr(model, 'module') else model 
	model_to_save.save_pretrained(strategy_model_dir)
	print('TRAIN done!')

In [15]:
metric = evaluate.load("squad")

In [14]:
def get_pred(dataloader, device, features, examples, record_loss=False, rd_0=False):
    if rd_0:
        config = BertConfig.from_pretrained(pretrain_model_dir, output_hidden_states=record_loss)
    else:
        config = BertConfig.from_pretrained(model_dir, output_hidden_states=record_loss)
    model = AutoModelForQuestionAnswering.from_config(config).to(device)
    
    test_loss = []
    model.eval()
    start_logits = []
    end_logits = []
    for batch in tqdm(dataloader, desc="Evaluating_pred"):
        batch = {key: value.to(device) for key, value in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            print(outputs)
            print(outputs.loss)
            test_loss.append(outputs.loss)

        start_logits.append(outputs.start_logits.cpu().numpy())
        end_logits.append(outputs.end_logits.cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(features)]
    end_logits = end_logits[: len(features)]

    if record_loss:
        test_loss /= len(dataloader.dataset)
        print('\nTest set: Average loss: {:.4f}\n'.format(test_loss))

    example_to_features = collections.defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples, desc="Computing metrics"):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [100]:
def logits_to_prob(start_logits, end_logits, features, batch_idx, examples, num_classes=False):
    
    probs = torch.zeros([len(batch_idx), 200])
    # deepAL+: probs = torch.zeros([len(data), len(np.unique(data.Y))])
    prob_dict = {}
    example_to_features = collections.defaultdict(list)
    max_answer_length = 30
    n_best = 20
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append((idx, batch_idx[idx]))
    
    for example in tqdm(examples, desc="Computing metrics"):
        example_id = example["id"]
        answers = []
        
        # Loop through all features associated with that example
        for (feature_index, i) in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answers.append(start_logit[start_index] + end_logit[end_index])

            if num_classes:
                if 1 < len(answers) < 200: # pad to same numbers of possible answers
                    zero_list = [0] * (200 - len(answers))
                    answers.extend(zero_list)
                elif len(answers) >= 200:
                    answers = answers[:200]
                probs[feature_index] = torch.tensor(answers)
            else:
                prob_dict[i] = torch.tensor(answers)

    if num_classes:
        return probs
    else:
        return prob_dict

In [99]:
def get_prob(dataloader, device, features, examples, rd):
    if rd == 1:
        model = AutoModelForQuestionAnswering.from_pretrained(pretrain_model_dir).to(device)
    else:
        model = AutoModelForQuestionAnswering.from_config(model_dir).to(device)

    # deepAL+: self.clf.eval()
    model.eval()
    idxs_start = 0
    prob_dict = {}
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating_prob"):
            batch = {key: value.to(device) for key, value in batch.items()}
            outputs = model(**batch)

            # matually create features batch
            data_len_batch = len(outputs.start_logits)
            idxs_end = idxs_start + data_len_batch
            batch_idx = list(range(idxs_start, idxs_end))
            batch_feat = features.select(batch_idx)
            idxs_start = idxs_end

            prob_dict.update(logits_to_prob(outputs.start_logits.cpu().numpy(), outputs.end_logits.cpu().numpy(), batch_feat, batch_idx, examples)) # num_classes=False
    
    for k, v in prob_dict.items():
        prob_dict[k] = F.softmax(v, dim=0)

    return prob_dict

In [18]:
def get_prob_dropout(dataloader, device, features, examples, n_drop=10):
    # deepAL+: self.clf.train()
    model = AutoModelForQuestionAnswering.from_pretrained(model_dir).to(device)
    model.train()
    # deepAL+: probs = torch.zeros([len(data), len(np.unique(data.Y))])
    # deepAL+: loader = DataLoader(data, shuffle=False, **self.params['loader_te_args'])
    prob_dict = {}
    # deepAL+: for i in range(n_drop):
    # deepAL+:     with torch.no_grad():
    # deepAL+:         for x, y, idxs in loader:
    # deepAL+:             x, y = x.to(self.device), y.to(self.device)
    # deepAL+:             out, e1 = self.clf(x)
    # deepAL+:             prob = F.softmax(out, dim=1)
    # deepAL+:             probs[idxs] += prob.cpu()
    for i in range(n_drop):
        start_logits = []
        end_logits = []
        for batch in tqdm(dataloader, desc="Evaluating_prob_dropout"):
            batch = {key: value.to(device) for key, value in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)

            start_logits.append(outputs.start_logits.cpu().numpy())
            end_logits.append(outputs.end_logits.cpu().numpy())

        start_logits = np.concatenate(start_logits)
        end_logits = np.concatenate(end_logits)
        start_logits = start_logits[: len(features)]
        end_logits = end_logits[: len(features)]

        example_to_features = collections.defaultdict(list)
        max_answer_length = 30
        n_best = 20
            
        for idx, feature in enumerate(features):
            example_to_features[feature["example_id"]].append(idx)

        n = 0
        for example in tqdm(examples):
            example_id = example["id"]
            answers = []

            # Loop through all features associated with that example
            for feature_index in example_to_features[example_id]:
                start_logit = start_logits[feature_index]
                end_logit = end_logits[feature_index]
                offsets = features[feature_index]["offset_mapping"]

                start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                for start_index in start_indexes:
                    for end_index in end_indexes:
                        # Skip answers that are not fully in the context
                        if offsets[start_index] is None or offsets[end_index] is None:
                            continue
                        # Skip answers with a length that is either < 0 or > max_answer_length
                        if (
                            end_index < start_index
                            or end_index - start_index + 1 > max_answer_length
                        ):
                            continue

                        answers.append(start_logit[start_index] + end_logit[end_index])

            if 1 < len(answers) < 200: # pad to same numbers of possible answers
                zero_list = [0] * (200 - len(answers))
                answers.extend(zero_list)
            elif len(answers) >= 200:
                answers = answers[:200]

            if len(answers) > 1:
                if feature_index not in prob_dict:
                    prob_dict[feature_index] = softmax(answers)
                else:
                    prob_dict[feature_index] += softmax(answers)
            elif example_to_features[example_id] != []:
                if feature_index not in prob_dict:
                    prob_dict[feature_index] = np.array([0])   

    for key in prob_dict.keys():
        prob_dict[key] /= n_drop

    return prob_dict

In [19]:
# need to be tested
def get_prob_dropout_split(dataloader, device, features, examples, n_drop=10):
    ## use tensor to save the answers
    model = AutoModelForQuestionAnswering.from_pretrained(model_dir).to(device)
    model.train()

    probs = torch.zeros([n_drop, len(dataloader.dataset), 200])
    
    for i in range(n_drop):
        start_logits = []
        end_logits = []
        with torch.no_grad():
            for batch in tqdm(dataloader, desc="Evaluating_prob_dropout"):
                batch = {key: value.to(device) for key, value in batch.items()}
                
                outputs = model(**batch)

                # matually create features batch
                data_len_batch = len(outputs.start_logits)
                idxs_end = idxs_start + data_len_batch
                batch_idx = list(range(idxs_start, idxs_end))
                batch_feat = features.select(batch_idx)
                idxs_start = idxs_end

                out = logits_to_prob(outputs.start_logits.cpu().numpy(), outputs.end_logits.cpu().numpy(), batch_feat, batch_idx, examples, num_classes=True)
                prob = F.softmax(out, dim=1)
                # deepAL+: probs[i][idxs] += F.softmax(out, dim=1).cpu()
                probs[i][batch_idx] += F.softmax(out, dim=1).cpu()
                # start_logits.append(outputs.start_logits.cpu().numpy())
                # end_logits.append(outputs.end_logits.cpu().numpy())

            # start_logits = np.concatenate(start_logits)
            # end_logits = np.concatenate(end_logits)
            # start_logits = start_logits[: len(features)]
            # end_logits = end_logits[: len(features)]

            # example_to_features = collections.defaultdict(list)
            # max_answer_length = 30
            # n_best = 20
                
            # for idx, feature in enumerate(features):
            #     example_to_features[feature["example_id"]].append(idx)

            # n = 0
            # for example in tqdm(examples, desc="Computing metrics"):
            #     example_id = example["id"]
            #     answers = []

            #     # Loop through all features associated with that example
            #     for feature_index in example_to_features[example_id]:
            #         start_logit = start_logits[feature_index]
            #         end_logit = end_logits[feature_index]
            #         offsets = features[feature_index]["offset_mapping"]

            #         start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            #         end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            #         for start_index in start_indexes:
            #             for end_index in end_indexes:
            #                 # Skip answers that are not fully in the context
            #                 if offsets[start_index] is None or offsets[end_index] is None:
            #                     continue
            #                 # Skip answers with a length that is either < 0 or > max_answer_length
            #                 if (
            #                     end_index < start_index
            #                     or end_index - start_index + 1 > max_answer_length
            #                 ):
            #                     continue

            #                 answers.append(start_logit[start_index] + end_logit[end_index])

                
            #         if 1 < len(answers) < 200: # pad to same numbers of possible answers
            #             zero_list = [0] * (200 - len(answers))
            #             answers.extend(zero_list)
            #         elif len(answers) >= 200:
            #             answers = answers[:200]

                    # probs[i][feature_index] += torch.tensor(softmax(answers))
    return probs

In [20]:
def get_embeddings(dataloader, device, features, examples, rd):
    if rd == 1:
        config = BertConfig.from_pretrained(pretrain_model_dir, output_hidden_states=True)
    else: 
        config = BertConfig.from_pretrained(model_dir, output_hidden_states=True)
    model = AutoModelForQuestionAnswering.from_config(config).to(device)

    model.eval()
    embeddings = torch.zeros([len(dataloader.dataset), model.config.to_dict()['hidden_size']])
    idxs_start = 0

    # deepAL+: with torch.no_grad():
    # deepAL+:     for x, y, idxs in loader:
    # deepAL+:         x, y = x.to(self.device), y.to(self.device)
    # deepAL+:         out, e1 = self.clf(x)
    # deepAL+:         embeddings[idxs] = e1.cpu()

    # TODO: change the loop sequence, needs a test
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating_prob"):
            batch = {key: value.to(device) for key, value in batch.items()}
            outputs = model(**batch)
            # print('len_output:', len(outputs)) # 4
            # print('outputs:', outputs) # (loss, start_logits, end_logits, hidden_states)

            hidden_states = outputs.hidden_states
            # print('len_hidden_states:', len(hidden_states)) # 13 # each one has: (batch_size, sequence_length, hidden_size)
            # # hidden_states[0] -> last hidden states
            # print('len_hidden_states[0]:', len(hidden_states[0])) # 8, 8, 4
            # print('len_hidden_states[0][0]:', len(hidden_states[0][0])) # 384, 384, 384 # tokens in each sequence
            # print('len_hidden_states[0][0][0]:', len(hidden_states[0][0][0])) # 768, 768, 768 # number of hidden units
            # print('hidden_states:', hidden_states) 

            # TODO: Question!!!!!
            embedding_of_last_layer = hidden_states[-1][:, 0, :] # [:, 0, :] -> to get [cls], but all the same
            # print(embedding_of_last_layer[0][0])
            idxs_end = idxs_start + len(hidden_states[0])
            # print(idxs_start)
            # print(idxs_end)
            embeddings[idxs_start:idxs_end] = embedding_of_last_layer.cpu()
            idxs_start = idxs_end
        
    return embeddings 

In [20]:
def get_grad_embeddings(dataloader, device, features, examples, rd):
    if rd == 1:
        config = BertConfig.from_pretrained(pretrain_model_dir, output_hidden_states=True)
    else: 
        config = BertConfig.from_pretrained(model_dir, output_hidden_states=True)
    model = AutoModelForQuestionAnswering.from_config(config).to(device)
    model.eval()

    # deepAL+: nLab = self.params['num_class']
    nLab = 200
    embDim = model.config.to_dict()['hidden_size']
    embeddings = np.zeros([len(dataloader.dataset), embDim * nLab])

    prob_dict = []
    data_len_start = 0

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating_prob"):
            batch = {key: Variable(value.to(device)) for key, value in batch.items()}
                
            # deepAL+: out, e1 = self.clf(x)
            outputs = model(**batch)
            # deepAL+: e1 = e1.data.cpu().numpy()
            hidden_states = outputs.hidden_states
            embedding_of_last_layer = hidden_states[-1][:, 0, :]
            embedding_of_last_layer = embedding_of_last_layer.data.cpu().numpy()

            # matually create features batch
            data_len_batch = len(outputs.start_logits)
            data_len_end = data_len_start + data_len_batch
            batch_idx = list(range(data_len_start, data_len_end))
            batch_feat = features.select(batch_idx)
            data_len_start = data_len_end

            # deepAL+: batchProbs = F.softmax(out, dim=1).data.cpu().numpy()
            # deepAL+: maxInds = np.argmax(batchProbs, 1)
            start_logits = outputs.start_logits.cpu().numpy()
            end_logits = outputs.end_logits.cpu().numpy()

            probs = torch.zeros([len(batch_idx), 200])
            example_to_features = collections.defaultdict(list)
            max_answer_length = 30
            n_best = 20
            for idx, feature in enumerate(features):
                example_to_features[feature["example_id"]].append((idx, batch_idx[idx]))
            
            for example in tqdm(examples, desc="Computing metrics"):
                example_id = example["id"]
                answers = []
                
                # Loop through all features associated with that example
                for (feature_index, i) in example_to_features[example_id]:
                    start_logit = start_logits[feature_index]
                    end_logit = end_logits[feature_index]
                    offsets = features[feature_index]["offset_mapping"]

                    start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
                    end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
                    for start_index in start_indexes:
                        for end_index in end_indexes:
                            # Skip answers that are not fully in the context
                            if offsets[start_index] is None or offsets[end_index] is None:
                                continue
                            # Skip answers with a length that is either < 0 or > max_answer_length
                            if (
                                end_index < start_index
                                or end_index - start_index + 1 > max_answer_length
                            ):
                                continue

                            answers.append(start_logit[start_index] + end_logit[end_index])

                    if 1 < len(answers) < 200: # pad to same numbers of possible answers
                        zero_list = [0] * (200 - len(answers))
                        answers.extend(zero_list)
                    elif len(answers) >= 200:
                        answers = answers[:200]
                    probs[feature_index] = torch.tensor(answers)

            batchProbs = F.softmax(probs, dim=1).data.cpu().numpy()
            maxInds = np.argmax(batchProbs, 1)

            for j in range(data_len_batch):
                for c in range(nLab):
                    if c == maxInds[j]:
                        embeddings[batch_idx[j]][embDim * c : embDim * (c+1)] = deepcopy(embedding_of_last_layer[j]) * (1 - batchProbs[j][c]) * -1.0
                    else:
                        embeddings[batch_idx[j]][embDim * c : embDim * (c+1)] = deepcopy(embedding_of_last_layer[j]) * (-1 * batchProbs[j][c]) * -1.0
            
            return embeddings

## utils.py

In [22]:
def get_unlabel_data(n_pool, labeled_idxs, train_dataset):
    unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
    unlabeled_data = train_dataset.select(indices=unlabeled_idxs)
    return unlabeled_idxs, unlabeled_data

In [98]:
def softmax(x): 
    """Compute softmax values for each sets of scores in x."""
    softmax_num = np.exp(x) / np.sum(np.exp(x), axis=0)
    return np.round(softmax_num, decimals=4)

In [24]:
from scipy import stats
from sklearn.metrics import pairwise_distances
import pdb

# kmeans ++ initialization
def init_centers(X, K):
    ind = np.argmax([np.linalg.norm(s, 2) for s in X])
    mu = [X[ind]]
    indsAll = [ind]
    centInds = [0.] * len(X)
    cent = 0
    print('#Samps\tTotal Distance')
    while len(mu) < K:
        if len(mu) == 1:
            D2 = pairwise_distances(X, mu).ravel().astype(float)
        else:
            newD = pairwise_distances(X, [mu[-1]]).ravel().astype(float)
            for i in range(len(X)):
                if D2[i] >  newD[i]:
                    centInds[i] = cent
                    D2[i] = newD[i]
        print(str(len(mu)) + '\t' + str(sum(D2)), flush=True)
        if sum(D2) == 0.0: pdb.set_trace()
        D2 = D2.ravel().astype(float)
        Ddist = (D2 ** 2)/ sum(D2 ** 2)
        customDist = stats.rv_discrete(name='custm', values=(np.arange(len(D2)), Ddist))
        ind = customDist.rvs(size=1)[0]
        while ind in indsAll: ind = customDist.rvs(size=1)[0]
        mu.append(X[ind])
        indsAll.append(ind)
        cent += 1
    return indsAll

## Query

In [25]:
def random_sampling_query(labeled_idxs, n, rd):
    return np.random.choice(np.where(labeled_idxs==0)[0], n, replace=False)

In [26]:
def margin_sampling_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=False,
		collate_fn=default_data_collator,
		batch_size=8,
	)
    # TODO: print for recording
    print('Margin querying starts!')
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_dict = get_prob(unlabeled_dataloader, device, unlabeled_features, examples, rd)
    # TODO: print for recording
    print('Got probability!')
    # deepAL+: probs_sorted, _ = probs.sort(descending=True)
    # deepAL+: uncertainties = probs_sorted[:, 0] - probs_sorted[:,1]
    uncertainties_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            sort_probs = np.sort(probs)[::-1] # This method returns a copy of the array, leaving the original array unchanged.
            uncertainties_dict[idx] = sort_probs[0] - sort_probs[1]
        elif idx:
            uncertainties_dict[idx] = np.array([0])

    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]] 
    sorted_uncertainties_list = sorted(uncertainties_dict.items(), key=lambda x: x[1], reverse=True)
    
    return unlabeled_idxs[[idx for (idx, uncertainties) in sorted_uncertainties_list[:n]]]

In [27]:
def least_confidence_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=False,
		collate_fn=default_data_collator,
		batch_size=8,
	)
    # TODO: print for recording
    print('LC querying starts!')
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_dict = get_prob(unlabeled_dataloader, device, unlabeled_features, examples)
    # TODO: print for recording
    print('Got probability!')
    # deepAL+: uncertainties = probs.max(1)[0]
    confidence_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            confidence_dict[idx] = max(probs)
        elif idx:
            confidence_dict[idx] = np.array([0])

    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]
    sorted_confidence_list = sorted(confidence_dict.items, key=lambda x: x[1])
    return unlabeled_idxs[[idx for (idx, confidence) in sorted_confidence_list[:n]]]

In [28]:
def var_ratio_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=False,
		collate_fn=default_data_collator,
		batch_size=8,
	)
    # TODO: print for recording
    print('Var Ratio querying starts!')
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    prob_dict = get_prob(unlabeled_dataloader, device, unlabeled_features, examples)
    # TODO: print for recording
    print('Got probability!')
    # deepAL+: preds = torch.max(probs, 1)[0]
    # deepAL+: uncertainties = 1.0 - preds
    confidence_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            confidence_dict[idx] = 1.0 - max(probs)
        elif idx:
            confidence_dict[idx] = np.array([0])

    # deepAL+: return unlabeled_idxs[uncertainties.sort(descending=True)[1][:n]]
    sorted_confidence_list = sorted(confidence_dict.items, key=lambda x: x[1], reverse=True)
    return unlabeled_idxs[[idx for (idx, confidence) in sorted_confidence_list[:n]]]
# comment for the same query as LC

In [29]:
def entropy_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=False,
		collate_fn=default_data_collator,
		batch_size=8,
	)
    # deepAL+: probs = self.predict_prob(unlabeled_data)
    # TODO: print for recording
    print('Entropy querying starts!')
    prob_dict = get_prob(unlabeled_dataloader, device, unlabeled_features, examples)
    # TODO: print for recording
    print('Got probability!')
    # deepAL+: log_probs = torch.log(probs)
    # deepAL+: uncertainties = (probs*log_probs).sum(1)
    entropy_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            log_probs = np.log(probs)
            entropy_dict[idx] = (probs*log_probs).sum()
        elif idx:
            entropy_dict[idx] = np.array([0])
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]
    sorted_entropy_list = sorted(entropy_dict.items(), key=lambda x: x[1])
    return unlabeled_idxs[[idx for (idx, entropy) in sorted_entropy_list[:n]]]

In [30]:
def margin_sampling_dropout_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
		unlabeled_data,
		shuffle=False,
		collate_fn=default_data_collator,
		batch_size=8,
	)
    # TODO: print for recording
    print('Margin dropout querying starts!')
    prob_dict = get_prob_dropout(unlabeled_dataloader, device, unlabeled_features, examples)
    # TODO: print for recording
    print('Got probability!')
    uncertainties_dict = {}
    for idx, probs in prob_dict.items():
        if len(probs) > 1: # if prob_dict['probs'] is not 0
            sort_probs = np.sort(probs)[::-1] # This method returns a copy of the array, leaving the original array unchanged.
            uncertainties_dict[idx] = sort_probs[0] - sort_probs[1]
        elif idx:
            uncertainties_dict[idx] = np.array([0])

    sorted_uncertainties_list = sorted(uncertainties_dict.items(), key=lambda x: x[1], reverse=True)
    return unlabeled_idxs[[idx for (idx, uncertainties) in sorted_uncertainties_list[:n]]]

In [31]:
def mean_std_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
  		unlabeled_data,
      shuffle=False,
      collate_fn=default_data_collator,
      batch_size=8,
    )
    # TODO: print for recording
    print('Mean STD querying starts!')
    # deepAL+: probs = self.predict_prob_dropout_split(unlabeled_data, n_drop=self.n_drop).numpy()
    probs = get_prob_dropout_split(unlabeled_dataloader, device, unlabeled_features, examples).numpy()
    # TODO: print for recording
    print('Got probability!')
    # deepAL+: sigma_c = np.std(probs, axis=0)
    sigma_c = np.std(probs, axis=0)
    # deepAL+: uncertainties = torch.from_numpy(np.mean(sigma_c, axis=-1))
    uncertainties = torch.from_numpy(np.mean(sigma_c, axis=-1)) # use tensor.sort() will sort the data and produce sorted indexes
    # deepAL+: return unlabeled_idxs[uncertainties.sort(descending=True)[1][:n]] # [1]: to get sorted data's indexes
    return unlabeled_idxs[uncertainties.sort(descending=True)[1][:n]]

In [32]:
def bayesian_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
      unlabeled_data,
      shuffle=False,
      collate_fn=default_data_collator,
      batch_size=8,
    )
    # deepAL+: probs = self.predict_prob_dropout_split(unlabeled_data, n_drop=self.n_drop)
    probs = get_prob_dropout_split(unlabeled_dataloader, device, unlabeled_features, examples)
    # deepAL+: pb = probs.mean(0)
    probs_mean = probs.mean(0)
    # deepAL+: entropy1 = (-pb*torch.log(pb)).sum(1)
    entropy1 = (-probs_mean*torch.log(probs_mean)).sum(1)
    # deepAL+: entropy2 = (-probs*torch.log(probs)).sum(2).mean(0)
    entropy2 = (-probs*torch.log(probs)).sum(2).mean(0)
    # deepAL+: uncertainties = entropy2 - entropy1
    uncertainties = entropy2 - entropy1
    # later on, we can use batch
    # deepAL+: return unlabeled_idxs[uncertainties.sort()[1][:n]]
    return unlabeled_idxs[uncertainties.sort()[1][:n]]

In [33]:
def bayesian_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
      unlabeled_data,
      shuffle=False,
      collate_fn=default_data_collator,
      batch_size=8,
    )
    # TODO: print for recording
    print('BALD querying starts!')
    probs = get_prob_dropout_split(unlabeled_dataloader, device, unlabeled_features, examples)
    # TODO: print for recording
    print('Got probability!')
    probs_mean = probs.mean(0)
    entropy1 = (-probs_mean*torch.log(probs_mean)).sum(1)
    entropy2 = (-probs*torch.log(probs)).sum(2).mean(0)
    uncertainties = entropy2 - entropy1
    # later on, we can use batch
    return unlabeled_idxs[uncertainties.sort()[1][:n]]

In [34]:
def mean_std_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
  		unlabeled_data,
      shuffle=False,
      collate_fn=default_data_collator,
      batch_size=8,
    )
    # TODO: print for recording
    print('Mean STD querying starts!')
    probs = get_prob_dropout_split(unlabeled_dataloader, device, unlabeled_features, examples).numpy()
    # TODO: print for recording
    print('Got probability!')
    sigma_c = np.std(probs, axis=0)
    uncertainties = torch.from_numpy(np.mean(sigma_c, axis=-1)) # use tensor.sort() will sort the data and produce sorted indexes
    return unlabeled_idxs[uncertainties.sort(descending=True)[1][:n]]

In [35]:
def kmeans_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(
      unlabeled_data,
      shuffle=False,
      collate_fn=default_data_collator,
      batch_size=8,
    )
    # deepAL+: embeddings = get_embeddings(unlabeled_data)
    embeddings = get_embeddings(unlabeled_dataloader, device, unlabeled_features, examples, rd)
    # deepAL+: embeddings = embeddings.numpy()
    embeddings = embeddings.numpy()
    # print(embeddings.shape)
    # deepAL+: cluster_learner = KMeans(n_clusters=n)
    cluster_learner = KMeans(n_clusters=n)
    # deepAL+: cluster_learner.fit(embeddings)
    cluster_learner.fit(embeddings)
    # deepAL+: cluster_idxs = cluster_learner.predict(embeddings)
    cluster_idxs = cluster_learner.predict(embeddings)
    # deepAL+: centers = cluster_learner.cluster_centers_[cluster_idxs]
    centers = cluster_learner.cluster_centers_[cluster_idxs]
    # deepAL+: dis = (embeddings - centers)**2
    dis = (embeddings - centers)**2
    # deepAL+: dis = dis.sum(axis=1)
    dis = dis.sum(axis=1)
    # deepAL+: q_idxs = np.array([np.arange(embeddings.shape[0])[cluster_idxs==i][dis[cluster_idxs==i].argmin()] for i in range(n)])
    q_idxs = np.array([np.arange(embeddings.shape[0])[cluster_idxs==i][dis[cluster_idxs==i].argmin()] for i in range(n)])

    # deepAL+: return unlabeled_idxs[q_idxs]
    return unlabeled_idxs[q_idxs]
    
    

In [36]:
def kcenter_greedy_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    # deepAL+: labeled_idxs, train_data = self.dataset.get_train_data()
    labeled_idxs_in_query = labeled_idxs.copy()
    # train_data = train_dataset
    train_dataloader = DataLoader(
      train_dataset,
      shuffle=False,
      collate_fn=default_data_collator,
      batch_size=8,
    )
    # deepAL+: embeddings = self.get_embeddings(train_data)
    embeddings = get_embeddings(train_dataloader, device, train_features, examples, rd)
    # deepAL+: embeddings = embeddings.numpy()
    embeddings = embeddings.numpy()
    # deepAL+: dist_mat = np.matmul(embeddings, embeddings.transpose())
    dist_mat = np.matmul(embeddings, embeddings.transpose())
    # deepAL+: sq = np.array(dist_mat.diagonal()).reshape(len(labeled_idxs), 1)
    sq = np.array(dist_mat.diagonal()).reshape(len(labeled_idxs_in_query), 1)
    # deepAL+: dist_mat *= -2
    dist_mat *= -2
    # deepAL+: dist_mat += sq
    dist_mat += sq
    # deepAL+: dist_mat += sq.transpose()
    dist_mat += sq.transpose()
    # deepAL+: dist_mat = np.sqrt(dist_mat)
    dist_mat = np.sqrt(dist_mat)

    # deepAL+: mat = dist_mat[~labeled_idxs, :][:, labeled_idxs]
    mat = dist_mat[~labeled_idxs_in_query, :][:, labeled_idxs_in_query]

    # deepAL+: for i in tqdm(range(n), ncols=100):
    for i in tqdm(range(n), ncols=100):
    # deepAL+:     mat_min = mat.min(axis=1)
        mat_min = mat.min(axis=1)
    # deepAL+:     q_idx_ = mat_min.argmax()
        q_idx_ = mat_min.argmax()
    # deepAL+:     q_idx = np.arange(self.dataset.n_pool)[~labeled_idxs][q_idx_]
        q_idx = np.arange(n_pool)[~labeled_idxs_in_query][q_idx_]
    # deepAL+:     labeled_idxs[q_idx] = True
        labeled_idxs_in_query[q_idx] = True
    # deepAL+:     mat = np.delete(mat, q_idx_, 0)
        mat = np.delete(mat, q_idx_, 0)
    # deepAL+:     mat = np.append(mat, dist_mat[~labeled_idxs, q_idx][:, None], axis=1)
        mat = np.append(mat, dist_mat[~labeled_idxs_in_query, q_idx][:, None], axis=1)
        
    # deepAL+: return np.arange(self.dataset.n_pool)[(self.dataset.labeled_idxs ^ labeled_idxs)]
    return np.arange(n_pool)[(labeled_idxs ^ labeled_idxs_in_query)]

In [37]:
def kcenter_greedy_PCA_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    # deepAL+: labeled_idxs, train_data = self.dataset.get_train_data()
    labeled_idxs_in_query = labeled_idxs.copy()
    # train_data = train_dataset
    train_dataloader = DataLoader(
      train_dataset,
      shuffle=False,
      collate_fn=default_data_collator,
      batch_size=8,
    )
    # deepAL+: embeddings = self.get_embeddings(train_data)
    embeddings = get_embeddings(train_dataloader, device, train_features, examples, rd)
    # deepAL+: embeddings = embeddings.numpy()
    embeddings = embeddings.numpy()
    # deepAL+: dist_mat = np.matmul(embeddings, embeddings.transpose())
    dist_mat = np.matmul(embeddings, embeddings.transpose())

    # deepAL+: downsampling embeddings if feature dim > 50
    # deepAL+: 
    if len(embeddings[0]) > 50:
        pca = PCA(n_components=50)
        embeddings = pca.fit_transform(embeddings)
    embeddings = embeddings.astype(np.float16)

    # deepAL+: sq = np.array(dist_mat.diagonal()).reshape(len(labeled_idxs), 1)
    sq = np.array(dist_mat.diagonal()).reshape(len(labeled_idxs_in_query), 1)
    # deepAL+: dist_mat *= -2
    dist_mat *= -2
    # deepAL+: dist_mat += sq
    dist_mat += sq
    # deepAL+: dist_mat += sq.transpose()
    dist_mat += sq.transpose()
    # deepAL+: dist_mat = np.sqrt(dist_mat)
    dist_mat = np.sqrt(dist_mat)

    # deepAL+: mat = dist_mat[~labeled_idxs, :][:, labeled_idxs]
    mat = dist_mat[~labeled_idxs_in_query, :][:, labeled_idxs_in_query]

    # deepAL+: for i in tqdm(range(n), ncols=100):
    for i in tqdm(range(n), ncols=100):
    # deepAL+:     mat_min = mat.min(axis=1)
        mat_min = mat.min(axis=1)
    # deepAL+:     q_idx_ = mat_min.argmax()
        q_idx_ = mat_min.argmax()
    # deepAL+:     q_idx = np.arange(self.dataset.n_pool)[~labeled_idxs][q_idx_]
        q_idx = np.arange(n_pool)[~labeled_idxs_in_query][q_idx_]
    # deepAL+:     labeled_idxs[q_idx] = True
        labeled_idxs_in_query[q_idx] = True
    # deepAL+:     mat = np.delete(mat, q_idx_, 0)
        mat = np.delete(mat, q_idx_, 0)
    # deepAL+:     mat = np.append(mat, dist_mat[~labeled_idxs, q_idx][:, None], axis=1)
        mat = np.append(mat, dist_mat[~labeled_idxs_in_query, q_idx][:, None], axis=1)
        
    # deepAL+: return np.arange(self.dataset.n_pool)[(self.dataset.labeled_idxs ^ labeled_idxs)]
    return np.arange(n_pool)[(labeled_idxs ^ labeled_idxs_in_query)]

In [39]:
def badge_query(n_pool, labeled_idxs, train_dataset, train_features, examples, device, n, rd):
    # deepAL+: unlabeled_idxs, unlabeled_data = self.dataset.get_unlabeled_data()
    unlabeled_idxs, unlabeled_data = get_unlabel_data(n_pool, labeled_idxs, train_dataset)
    unlabeled_features = train_features.select(unlabeled_idxs)
    unlabeled_dataloader = DataLoader(unlabeled_data,
                                      shuffle=False,
                                      collate_fn=default_data_collator,
                                      batch_size=8,
                                    )
    # deepAL+: gradEmbedding = self.get_grad_embeddings(unlabeled_data)
    gradEmbedding = get_grad_embeddings(unlabeled_dataloader, device, unlabeled_features, examples, rd)
    # deepAL+: chosen = init_centers(gradEmbedding, n)
    chosen = init_centers(gradEmbedding, n)
    # deepAL+: return unlabeled_idxs[chosen]
    return unlabeled_idxs[chosen]

# main.py

### seed and device

In [40]:
SEED = 4666
# os.environ['TORCH_HOME']='./basicmodel'
os.environ["CUDA_VISIBLE_DEVICES"] = str(5)

# fix random seed
np.random.seed(SEED)
torch.manual_seed(SEED)
# torch.backends.cudnn.enabled  = True
# torch.backends.cudnn.benchmark= True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### start experiment

In [41]:
iteration = args_input_iteration
model_batch = args_input_model_batch
num_train_epochs = 3

all_acc = []
acq_time = []

# Change "fp16_training" to True to support automatic mixed precision training (fp16)	
fp16_training = False

if fp16_training:
    !pip install accelerate==0.2.0
    from accelerate import Accelerator
    accelerator = Accelerator(fp16=True)
    device = accelerator.device

In [42]:
## generate initial labeled pool
n_pool = len(train_dataset)
labeled_idxs = np.zeros(n_pool, dtype=bool)

tmp_idxs = np.arange(n_pool)
np.random.shuffle(tmp_idxs)
labeled_idxs[tmp_idxs[:NUM_INIT_LB]] = True

run_0_labeled_idxs = np.arange(n_pool)[labeled_idxs]

## record acc performance 
acc = np.zeros(NUM_ROUND + 1) # quota/batch runs + run_0

## data
# eval_dataloader = DataLoader(
# 	val_dataset, 
# 	collate_fn=default_data_collator, 
# 	batch_size=8
# )

## print info
print(DATA_NAME)
print(STRATEGY_NAME)

## round 0 accuracy
# acc[0] = get_pred(eval_dataloader, device, val_features, squad['validation'], rd_0=True)['f1']
# acc[0] = 77.96450701 # init=4000
acc[0] = 13.8252 # init=100

print('Round 0\ntesting accuracy {}'.format(acc[0]))
print('\n')

SQuAD
KMeansSampling
Round 0
testing accuracy 13.8252




# query workspace

### test: query 5 data from 20 unlabeled_data

In [43]:
# get unlable data
unlabeled_idxs = np.arange(n_pool)[~labeled_idxs]
# smaller data
unlabeled_idxs_20 = unlabeled_idxs[20:40]
unlabeled_data_20 = train_dataset.select(unlabeled_idxs_20)
len(unlabeled_data_20)

20

In [44]:
unlabeled_features_20 = train_features.select(unlabeled_idxs_20)
unlabeled_dataloader_20 = DataLoader(
		unlabeled_data_20,
		shuffle=False,
		collate_fn=default_data_collator,
		batch_size=8,
	)

In [46]:
dataloader = unlabeled_dataloader_20
features = unlabeled_features_20
examples = squad['train']
rd = 1

In [102]:
get_prob_dict = get_prob(unlabeled_dataloader_20, device, unlabeled_features_20, squad['train'], rd)

Evaluating_prob:   0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4000 [00:00<?, ?it/s]

In [103]:
get_prob_2_dict = get_prob_2(dataloader, device, features, examples, rd)

Evaluating_prob:   0%|          | 0/3 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/4000 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/4000 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/4000 [00:00<?, ?it/s]

In [107]:
for k in get_prob_dict.keys():
    print(np.array_equal(get_prob_dict[k], get_prob_2_dict[k]))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [104]:
get_prob_dict[16]

array([0.0932, 0.0314, 0.0264, 0.0238, 0.0128, 0.0081, 0.0064, 0.0706,
       0.0238, 0.02  , 0.0097, 0.0062, 0.0051, 0.0048, 0.0585, 0.0304,
       0.0149, 0.008 , 0.0064, 0.0049, 0.1007, 0.0215, 0.0082, 0.0072,
       0.0052, 0.0582, 0.0124, 0.0047, 0.003 , 0.054 , 0.0115, 0.0044,
       0.0039, 0.0028, 0.0037, 0.0288, 0.0087, 0.0061, 0.0023, 0.0021,
       0.0018, 0.0015, 0.0014, 0.0273, 0.0058, 0.0022, 0.002 , 0.0014,
       0.0134, 0.0045, 0.0038, 0.0034, 0.0018, 0.0012, 0.0009, 0.0045,
       0.0017, 0.0086, 0.0029, 0.0024, 0.0022, 0.0012, 0.0008, 0.0148,
       0.0044, 0.0031, 0.0012, 0.0011, 0.0008, 0.0007, 0.0082, 0.0028,
       0.0023, 0.0021, 0.0011, 0.0009, 0.0017, 0.0008, 0.0006, 0.0005,
       0.0005, 0.0027, 0.001 , 0.007 , 0.0024, 0.002 , 0.0016, 0.0006,
       0.0006, 0.0005, 0.0005, 0.0056, 0.0019, 0.0016, 0.0014, 0.0008,
       0.0005, 0.005 , 0.0017, 0.0014, 0.0007, 0.0004, 0.0004, 0.0003],
      dtype=float32)

In [105]:
get_prob_2_dict[16]

array([0.0932, 0.0314, 0.0264, 0.0238, 0.0128, 0.0081, 0.0064, 0.0706,
       0.0238, 0.02  , 0.0097, 0.0062, 0.0051, 0.0048, 0.0585, 0.0304,
       0.0149, 0.008 , 0.0064, 0.0049, 0.1007, 0.0215, 0.0082, 0.0072,
       0.0052, 0.0582, 0.0124, 0.0047, 0.003 , 0.054 , 0.0115, 0.0044,
       0.0039, 0.0028, 0.0037, 0.0288, 0.0087, 0.0061, 0.0023, 0.0021,
       0.0018, 0.0015, 0.0014, 0.0273, 0.0058, 0.0022, 0.002 , 0.0014,
       0.0134, 0.0045, 0.0038, 0.0034, 0.0018, 0.0012, 0.0009, 0.0045,
       0.0017, 0.0086, 0.0029, 0.0024, 0.0022, 0.0012, 0.0008, 0.0148,
       0.0044, 0.0031, 0.0012, 0.0011, 0.0008, 0.0007, 0.0082, 0.0028,
       0.0023, 0.0021, 0.0011, 0.0009, 0.0017, 0.0008, 0.0006, 0.0005,
       0.0005, 0.0027, 0.001 , 0.007 , 0.0024, 0.002 , 0.0016, 0.0006,
       0.0006, 0.0005, 0.0005, 0.0056, 0.0019, 0.0016, 0.0014, 0.0008,
       0.0005, 0.005 , 0.0017, 0.0014, 0.0007, 0.0004, 0.0004, 0.0003],
      dtype=float32)

In [194]:
from torch.autograd import Variable
import torch.nn.functional as F
from copy import deepcopy
# prob_dict = get_prob(unlabeled_dataloader_20, device, unlabeled_features_20, squad['train'], rd)
dataloader = unlabeled_dataloader_20
features = unlabeled_features_20
examples = squad['train']
rd = 1

if rd == 1:
    config = BertConfig.from_pretrained(pretrain_model_dir, output_hidden_states=True)
else: 
    config = BertConfig.from_pretrained(model_dir, output_hidden_states=True)
model = AutoModelForQuestionAnswering.from_config(config).to(device)
model.eval()

# deepAL+: nLab = self.params['num_class']
nLab = 200
embDim = model.config.to_dict()['hidden_size']
embeddings = np.zeros([len(dataloader.dataset), embDim * nLab])

prob_dict = []
data_len_start = 0

with torch.no_grad():
    for batch in tqdm(dataloader, desc="Evaluating_prob"):
        batch = {key: Variable(value.to(device)) for key, value in batch.items()}
            
        # deepAL+: out, e1 = self.clf(x)
        outputs = model(**batch)
        # deepAL+: e1 = e1.data.cpu().numpy()
        hidden_states = outputs.hidden_states
        embedding_of_last_layer = hidden_states[-2][:, 0, :]
        embedding_of_last_layer = embedding_of_last_layer.data.cpu().numpy()

        # idxs_end = idxs_start + len(hidden_states[0])
        # # deepAL+: embeddings[idxs] = e1.cpu()
        # embeddings[idxs_start:idxs_end] = embedding_of_last_layer.cpu()
        # idxs_start = idxs_end

        # matually create features batch
        data_len_batch = len(outputs.start_logits)
        data_len_end = data_len_start + data_len_batch
        batch_idx = list(range(data_len_start, data_len_end))
        batch_feat = unlabeled_features_20.select(batch_idx)
        data_len_start = data_len_end

        # deepAL+: batchProbs = F.softmax(out, dim=1).data.cpu().numpy()
        # deepAL+: maxInds = np.argmax(batchProbs, 1)
        out = logits_to_prob(outputs.start_logits.cpu().numpy(), outputs.end_logits.cpu().numpy(), batch_feat, batch_idx, examples, num_classes=True)
        batchProbs = F.softmax(out, dim=1).data.cpu().numpy()
        maxInds = np.argmax(batchProbs, 1)

        for j in range(data_len_batch):
            for c in range(nLab):
                if c == maxInds[j]:
                    embeddings[batch_idx[j]][embDim * c : embDim * (c+1)] = deepcopy(embedding_of_last_layer[j]) * (1 - batchProbs[j][c]) * -1.0
                else:
                    embeddings[batch_idx[j]][embDim * c : embDim * (c+1)] = deepcopy(embedding_of_last_layer[j]) * (-1 * batchProbs[j][c]) * -1.0
        # prob_dict.append(maxInds)


Evaluating_prob:   0%|          | 0/3 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/4000 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/4000 [00:00<?, ?it/s]

Computing metrics:   0%|          | 0/4000 [00:00<?, ?it/s]

In [196]:
embeddings.shape

(20, 153600)

In [198]:
# chosen = init_centers(gradEmbedding, n)
chosen = init_centers(embeddings, 5)
unlabeled_idxs[chosen]

#Samps	Total Distance
1	719.9505246232893
2	575.2702332799374
3	502.7837294584695
4	480.8306760112471


array([ 4, 12, 11,  1,  3])

In [111]:
int(100 / 30)

3